# BSP Tree 

# Advanced Python Course 
## Mobi Heidelberg WS 2021/22
### by Christian Fufezan 

christian@fufezan.net

https://fufezan.net

<img src="./images/cc.png" alt="drawing" width="200" style="float: left;"/>

In [1]:
import csv

In [2]:
import csv
tree = {}
cities = []
lookup = []
for ldict in csv.DictReader(open("../data/cities.csv")):
    cities.append((float(ldict['lat']), float(ldict['lng']), ldict['city'], len(lookup)))
    lookup.append(ldict)

In [3]:
len(cities)

41001

In [4]:
cities.sort(key=lambda x: x[0])

In [5]:
cities[:10]

[(-54.9341, -67.6109, 'Puerto Williams', 40252),
 (-54.8072, -68.3044, 'Ushuaia', 5803),
 (-54.2806, -36.508, 'Grytviken', 40958),
 (-53.7914, -67.699, 'Río Grande', 16008),
 (-53.2956, -70.3687, 'Porvenir', 37422),
 (-53.1627, -70.9081, 'Punta Arenas', 4158),
 (-52.65, -71.4666, 'Balsadero Río Verde', 40921),
 (-51.7263, -72.5062, 'Puerto Natales', 20742),
 (-51.7, -57.85, 'Stanley', 854),
 (-51.6333, -69.2333, 'Río Gallegos', 5008)]

In [6]:
cities.sort(key=lambda x: x[1])

In [7]:
cities[:10]

[(70.9566, -179.59, 'Zvëzdnyy', 40992),
 (66.3221, -179.1837, 'Egvekinot', 40847),
 (-14.2933, -178.1583, 'Leava', 10144),
 (-43.951, -176.561, 'Waitangi', 9837),
 (-13.2825, -176.1736, 'Mata-Utu', 865),
 (-21.1347, -175.2083, 'Nuku‘alofa', 834),
 (-18.6496, -173.9833, 'Neiafu', 40497),
 (64.4235, -173.2258, 'Provideniya', 40835),
 (-13.5196, -172.6378, 'Asau', 10183),
 (-13.4513, -172.4018, 'Safotu', 10187)]

In [8]:
lookup[40992]

OrderedDict([('city', 'Zvëzdnyy'),
             ('lat', '70.9566'),
             ('lng', '-179.59'),
             ('country', 'Russia'),
             ('iso3', 'RUS'),
             ('local_name', 'Chukotskiy Avtonomnyy Okrug'),
             ('population', '10.0'),
             ('continent', 'Europe')])

In [9]:
import math

def split_cities(cities=None, split_number = 0, root = None):
    """Splits list of cities depending on split_number by 
    either longitude or latidude array

    Args:
        cities (list of tuples): List of cities in the following format
            [
                (70.9566, -179.59, 'Zvëzdnyy', 40992),
                (66.3221, -179.1837, 'Egvekinot', 40847),
                (-14.2933, -178.1583, 'Leava', 10144),
                ...
            ]
        split_number (int): Defines direction to split city array during recursion.
        root (dict): Dictionary to be used for new split, on the leaf level, this is 
             only a list of cities 

    Returns:
        dict: root
    """
    direction = split_number % 2 # is index position
    cities.sort(key=lambda x: x[direction])
    split_pos = math.floor(len(cities) / 2)
    city_block1 = cities[:split_pos]
    city_block2 = cities[split_pos:]
    if len(city_block1) > 10:
        root_1 = split_cities(cities=city_block1.copy(), split_number=split_number + 1, root = {})
    else:
        root_1 = city_block1

    if len(city_block2) > 10:
        root_2 = split_cities(cities=city_block2.copy(), split_number=split_number + 1, root = {})
    else:
        root_2 = city_block2
    return {
        "direction": direction,
        "left" : {
            "lower_boundary" : city_block1[0],
            "upper_boundary" : city_block1[-1],
            "root" : root_1, 
        },
        "right" : {
            "lower_boundary": city_block2[0],
            "upper_boundary": city_block2[-1],
            "root": root_2
        }
    }


root = split_cities(cities=cities, split_number=0, root = {})


In [10]:

display(root.keys())
root['direction']

dict_keys(['direction', 'left', 'right'])

0

In [11]:

def find_closest_cities(point=None):
    if point is None:
        point = (0,0)
    current_root = root
    while True:
        direction = current_root['direction']
        if current_root['left']['lower_boundary'][direction] <= point[direction] <= current_root['left']['upper_boundary'][direction]:
            current_root = current_root['left']['root']
        elif current_root['right']['lower_boundary'][direction] <= point[direction] <= current_root['right']['upper_boundary'][direction]:
            current_root = current_root['right']['root']
        if isinstance(current_root, list):
            
            break 
    return current_root


In [12]:
%%timeit
heidelberg = (49.416667, 8.716667)
find_closest_cities(point=heidelberg)

7.31 µs ± 17.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
